 <div style="background: linear-gradient(to right, #434343, #000000); /* Dark gradient */
             border-left: 6px solid #fbc02d; /* Gold border */
             padding: 25px 30px;
             border-radius: 10px;
             box-shadow: 0 8px 20px rgba(0, 0, 0, 0.4);
             margin-bottom: 25px;
             animation: slideIn 0.8s ease-out;">
 <style>
   @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } }
   .custom-div-dark h1 { color: #ffffff; margin-bottom: 15px; font-family: 'Helvetica Neue', Arial, sans-serif; font-weight: 300; font-size: 2.5em; border-bottom: 1px solid #fbc02d; padding-bottom: 10px; }
   .custom-div-dark p, .custom-div-dark ul { line-height: 1.7; color: #f5f5f5; font-size: 1.1em; }
   .custom-div-dark ul { padding-left: 25px; margin-top: 15px; }
   .custom-div-dark li { margin-bottom: 8px; }
   .custom-div-dark code { background-color: #333; color: #fbc02d; padding: 2px 5px; border-radius: 4px; }
   .assumption-note-dark { background-color: #424242; border-left: 4px solid #f9a825; padding: 12px; margin: 20px 0; border-radius: 4px; font-style: italic; color: #eeeeee; }
 </style>
 <div class="custom-div-dark">

 <h1>🏁 Synthèse Décisionnelle pour l'Investissement</h1>

 **Objectif :** Ce notebook final synthétise les conclusions des quatre analyses précédentes (`visualisation`, `strategique`, `micro`, `rentabilite`). Il est conçu pour fournir une **réponse directe** à Alexandre Dubois, en identifiant les opportunités d'investissement les plus prometteuses basées *exclusivement* sur les données DVF.

 **Méthodologie de Synthèse :**
 1.  **Identification des Départements Stratégiques :** Nous ré-exécutons l'analyse de potentiel (Prix vs Évolution, Besoin III.1) pour filtrer les départements qui combinent **prix abordable**, **croissance positive** et **liquidité** (volume élevé).
 2.  **Comparaison Directe (Rhône vs Cibles) :** Nous répondons à la question d'Alexandre en comparant les KPI du Rhône (69) à ceux des départements cibles identifiés.
 3.  **Zoom "Micro" (Pépites Communales) :** Nous utilisons le "chargement chirurgical" (Besoin III.3) sur les départements cibles pour identifier les **communes** spécifiques présentant le meilleur potentiel d'appréciation.
 4.  **Simulation de Rentabilité :** Nous appliquons le simulateur de rendement (Besoin III.2) à ces communes cibles pour estimer ce qu'Alexandre peut espérer y gagner.
 5.  **Le Verdict Final :** Nous présentons un "Top 10" clair des opportunités d'investissement, accompagné d'une recommandation finale et des **limites cruciales** de l'analyse.

 </div>
 </div>

 <div style="background: linear-gradient(to right, #e0f7fa, #e0f2f7); /* Soft cyan gradient */
             border-left: 4px solid #80deea; /* Lighter, thinner cyan border */
             padding: 18px 20px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
             margin-bottom: 20px; animation: slideIn 0.7s ease-out;">
 <style> @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } }
   .custom-div-cyan h2 { color: #0097a7; margin-bottom: 12px; } .custom-div-cyan p { line-height: 1.6; } </style>
 <div class="custom-div-cyan">

 ## ⚙️ 1. Configuration et Fonctions Utilitaires Essentielles

 **Objectif :** Importer les bibliothèques et redéfinir les 3 fonctions clés de notre pipeline (`clean_dvf_data`, `load_specific_cities_data`, `calculate_yields`) pour assurer la cohérence et l'autonomie de cette synthèse.

 </div>
 </div>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
import os
from tqdm.auto import tqdm
import gc
from datetime import datetime
import io
import base64

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)
px.defaults.template = "plotly_white"

# --- 1. FONCTION DE NETTOYAGE (de visualisation.ipynb / analyse_micro.ipynb) ---
def clean_dvf_data(df):
    if df.empty: return pd.DataFrame()
    print(f"   -> Nettoyage de {len(df):,} lignes brutes...", end=" ")
    initial_rows = len(df)
    df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')
    cols_to_numeric = ['valeur_fonciere', 'surface_reelle_bati', 'nombre_pieces_principales']
    for col in cols_to_numeric:
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce')
    
    cols_critiques = ['date_mutation', 'valeur_fonciere', 'surface_reelle_bati', 'type_local', 'code_departement']
    if 'code_commune' in df.columns: cols_critiques.append('code_commune') # Requis pour micro
        
    cols_a_verifier = [col for col in cols_critiques if col in df.columns]
    df = df.dropna(subset=cols_a_verifier)
    
    df = df[df['valeur_fonciere'].between(1000, 20_000_000)]
    df = df[df['surface_reelle_bati'].between(10, 1000)]
    if 'nombre_pieces_principales' in df.columns:
      df = df[df['nombre_pieces_principales'].between(1, 20)]
    
    # Pour la synthèse, on se concentre sur l'habitation
    df = df[df['type_local'].isin(['Appartement', 'Maison'])]
    
    df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']
    
    # Filtrage outliers prix/m2 plus robuste (quantile)
    if not df.empty:
        p_01 = df['prix_m2'].quantile(0.01)
        p_99 = df['prix_m2'].quantile(0.99)
        df = df[df['prix_m2'].between(max(100, p_01), min(30000, p_99))]
    
    df['annee'] = df['date_mutation'].dt.year
    df['trimestre'] = df['date_mutation'].dt.to_period('Q').astype(str)
    
    final_rows = len(df)
    print(f"Terminé. {final_rows:,} transactions valides restantes.")
    return df

# --- 2. FONCTION DE CHARGEMENT (de analyse_strategique.ipynb / analyse_micro.ipynb) ---
BASE_CSV_PATH = './'
FILE_NAMES_CSV = [f"dvf_geolocalisees_{year}.csv" for year in range(2021, 2026)]
FILE_PATHS_CSV = [os.path.join(BASE_CSV_PATH, f) for f in FILE_NAMES_CSV if os.path.exists(os.path.join(BASE_CSV_PATH, f))]
COLS_FOR_CITY = ['date_mutation', 'valeur_fonciere', 'code_commune', 'nom_commune', 'code_departement',
                 'surface_reelle_bati', 'nombre_pieces_principales', 'type_local', 'latitude', 'longitude']
DTYPES_FOR_CITY = {'valeur_fonciere': 'float32', 'code_commune': 'str', 'nom_commune': 'str', 'code_departement': 'str',
                   'surface_reelle_bati': 'float32', 'nombre_pieces_principales': 'float32',
                   'type_local': 'category', 'latitude': 'float32', 'longitude': 'float32'}

def load_specific_cities_data(city_codes, file_paths=FILE_PATHS_CSV, use_cols=COLS_FOR_CITY, dtypes=DTYPES_FOR_CITY):
    print(f"🔬 Chargement chirurgical des données pour les communes : {city_codes}...")
    all_city_chunks = []
    if not file_paths: print("❌ Aucun fichier CSV source trouvé."); return pd.DataFrame()
    city_codes_str = [str(c) for c in city_codes]

    for file_path in tqdm(file_paths, desc="Lecture CSV (Communes)", leave=False):
        try:
            chunk_iter = pd.read_csv(file_path, usecols=lambda col: col in use_cols, dtype=dtypes,
                                     chunksize=100_000, low_memory=False, on_bad_lines='skip')
            for chunk in chunk_iter:
                if 'code_commune' in chunk.columns:
                    chunk_filtered = chunk[chunk['code_commune'].astype(str).isin(city_codes_str)]
                    if not chunk_filtered.empty: all_city_chunks.append(chunk_filtered)
        except Exception as e: print(f"\n⚠️ Erreur lecture {file_path}: {e}")

    if not all_city_chunks: print("❌ Aucune donnée trouvée."); return pd.DataFrame()
    
    df_cities_raw = pd.concat(all_city_chunks, ignore_index=True)
    df_cities_clean = clean_dvf_data(df_cities_raw)
    
    print("   -> Optimisation mémoire finale...")
    for col in df_cities_clean.select_dtypes(include=['object', 'category']).columns:
         if col not in ['trimestre']: df_cities_clean[col] = df_cities_clean[col].astype('category')
    gc.collect()
    print(f"✅ Chargement et nettoyage terminés pour {city_codes}.")
    return df_cities_clean

# --- 3. FONCTION DE CALCUL DE RENDEMENT (de analyse_rentabilite.ipynb) ---
def calculate_yields(purchase_price, surface_m2, rent_m2_monthly,
                     charges_pct=15.0, tax_fonciere_pct=8.0, vacance_pct=5.0,
                     gestion_pct=7.0, gli_pct=3.0, reparations_pct=2.0):
    if purchase_price <= 0 or surface_m2 <= 0 or rent_m2_monthly <= 0:
        return {'rendement_brut_annuel_pct': 0, 'rendement_net_annuel_pct': 0}
    
    annual_rent_hc = rent_m2_monthly * surface_m2 * 12
    gross_yield_pct = (annual_rent_hc / purchase_price) * 100 if purchase_price > 0 else 0
    
    total_charges_pct = (charges_pct + tax_fonciere_pct + vacance_pct + 
                         gestion_pct + gli_pct + reparations_pct)
    total_charges_annual = (annual_rent_hc * total_charges_pct / 100)
    net_income_annual = annual_rent_hc - total_charges_annual
    net_yield_pct = (net_income_annual / purchase_price) * 100 if purchase_price > 0 else 0

    return {'rendement_brut_annuel_pct': round(gross_yield_pct, 2),
            'rendement_net_annuel_pct': round(net_yield_pct, 2)}

# --- 4. FONCTION DE SIMULATION DE LOYER (de analyse_rentabilite.ipynb) ---
def simulate_rent_m2(median_price_m2, gross_yield_assumption_percent=5.0):
    if median_price_m2 <= 0 or gross_yield_assumption_percent <= 0: return 0
    return (median_price_m2 * gross_yield_assumption_percent / 100) / 12

print("\n✅ Toutes les fonctions utilitaires sont prêtes.")

<div style="background: linear-gradient(to right, #e8f5e9, #f1f8e9); /* Soft green gradient */
             border-left: 4px solid #a5d6a7; /* Lighter, thinner green border */
             padding: 18px 20px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
             margin-bottom: 20px; animation: slideIn 0.7s ease-out;">
 <style> @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } }
   .custom-div-green h2 { color: #388e3c; margin-bottom: 12px; } .custom-div-green p { line-height: 1.6; } </style>
 <div class="custom-div-green">

 ## 🗺️ Étape 1 : Identification des Départements Stratégiques (Besoin III.1)

 **Objectif :** Identifier les départements français qui correspondent le mieux au profil d'Alexandre :
 * **Abordables :** Prix/m² inférieur à la médiane nationale (filtrée).
 * **En Croissance :** Évolution positive des prix sur les 5 dernières années.
 * **Liquides :** Volume de transactions élevé (minimum 50 000 ventes sur 5 ans).

 Nous chargeons `df_clean.parquet` et appliquons ce filtre.

 </div>
 </div>

In [ ]:
PARQUET_FILE_PATH = 'dvf_clean_2021-2025.parquet'
df_macro = pd.DataFrame()
if os.path.exists(PARQUET_FILE_PATH):
    print(f"✅ Chargement des données Macro depuis '{PARQUET_FILE_PATH}'...")
    df_macro = pd.read_parquet(PARQUET_FILE_PATH)
    print(f"   -> {len(df_macro):,} transactions chargées.")
else:
    print(f"❌ ERREUR: Le fichier Parquet '{PARQUET_FILE_PATH}' est introuvable.")
    print("   Veuillez exécuter le notebook 'visualisation.ipynb' d'abord.")

df_top_depts = pd.DataFrame() # Initialisation pour les cellules suivantes

if not df_macro.empty:
    print("Calcul du potentiel de croissance par département...")
    
    # S'assurer que 'annee' est numérique pour min/max
    df_macro['annee'] = pd.to_numeric(df_macro['annee'], errors='coerce').dropna().astype(int)
    start_year = df_macro['annee'].min()
    end_year = df_macro['annee'].max()

    # 1. Calculer les KPIs par département
    median_prices = df_macro[df_macro['annee'].isin([start_year, end_year])].groupby(['code_departement', 'annee'])['prix_m2'].median().unstack()
    total_volume = df_macro.groupby('code_departement').size().reset_index(name='Volume_Total_5Ans')
    
    analysis_data = pd.merge(median_prices, total_volume, on='code_departement')
    analysis_data = analysis_data.dropna() # Garder slmt depts avec données début/fin

    # Renommer les colonnes numériques
    col_start_year_str = f'Prix_m2_{start_year}'
    col_end_year_str = f'Prix_m2_{end_year}'
    analysis_data = analysis_data.rename(columns={start_year: col_start_year_str, end_year: col_end_year_str})

    # Calculer l'évolution
    analysis_data['Evolution_Pct_5Ans'] = ((analysis_data[col_end_year_str] - analysis_data[col_start_year_str]) / analysis_data[col_start_year_str]) * 100
    
    # 2. Appliquer les filtres 
    PRIX_MAX_ABORDABLE = 4000 # Prix/m² max pour un premier investissement (arbitraire, ajustable)
    VOLUME_MIN_LIQUIDE = 50000 # Nb min de transactions sur 5 ans
    
    df_top_depts = analysis_data[
        (analysis_data[col_end_year_str] <= PRIX_MAX_ABORDABLE) &
        (analysis_data['Evolution_Pct_5Ans'] > 0) & # Croissance positive
        (analysis_data['Volume_Total_5Ans'] >= VOLUME_MIN_LIQUIDE)
    ].sort_values('Evolution_Pct_5Ans', ascending=False)
    
    df_top_depts = df_top_depts.reset_index() # Mettre 'code_departement' en colonne
    
    print("\n--- ✅ Départements Cibles Identifiés (Abordables, Liquides, en Croissance) ---")
    display(df_top_depts[['code_departement', col_end_year_str, 'Evolution_Pct_5Ans', 'Volume_Total_5Ans']].head(10))

 <div style="background: linear-gradient(to right, #f3e5f5, #fce4ec); /* Soft purple/pink gradient */
             border-left: 4px solid #ce93d8; /* Lighter, thinner purple border */
             padding: 18px 20px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
             margin-bottom: 20px; animation: slideIn 0.7s ease-out;">
 <style> @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } }
   .custom-div-purple h2 { color: #7b1fa2; margin-bottom: 12px; } .custom-div-purple p { line-height: 1.6; } </style>
 <div class="custom-div-purple">

 ## 📊 Étape 2 : Comparaison Directe (Rhône vs. Cibles)

 **Objectif :** Répondre à la question d'Alexandre : "Puis-je investir dans ma ville (Rhône) par rapport aux autres ?"

 **Méthode :** Nous extrayons les KPIs du département 69 (Rhône) de notre analyse et les comparons au profil moyen de nos départements cibles.

 </div>
 </div>

In [ ]:
if not analysis_data.empty and not df_top_depts.empty:
    kpi_rhone = analysis_data[analysis_data['code_departement'] == '69']
    
    # Calculer les moyennes des cibles
    avg_cibles = {
        'Prix Moyen Cibles': df_top_depts[col_end_year_str].mean(),
        'Évolution Moyenne Cibles': df_top_depts['Evolution_Pct_5Ans'].mean(),
        'Volume Moyen Cibles': df_top_depts['Volume_Total_5Ans'].mean()
    }
    
    # Affichage de la comparaison
    print("\n--- Comparaison Stratégique : Rhône (69) vs. Départements Cibles ---")
    if not kpi_rhone.empty:
        kpi_rhone_data = kpi_rhone.iloc[0]
        compare_data = {
            'Indicateur': ['Prix/m² Récent', 'Évolution 5 Ans (%)', 'Volume 5 Ans'],
            'Rhône (69)': [
                f"{kpi_rhone_data[col_end_year_str]:,.0f} €",
                f"{kpi_rhone_data['Evolution_Pct_5Ans']:.2f}%",
                f"{kpi_rhone_data['Volume_Total_5Ans']:,}"
            ],
            'Moyenne Cibles': [
                f"{avg_cibles['Prix Moyen Cibles']:,.0f} €",
                f"{avg_cibles['Évolution Moyenne Cibles']:.2f}%",
                f"{avg_cibles['Volume Moyen Cibles']:,}"
            ]
        }
        display(pd.DataFrame(compare_data).set_index('Indicateur'))

<div style="background: linear-gradient(to right,#f3e5f5, #fce4ec); /* Soft green gradient */
             border-left: 4px solid #a5d6a7; /* Lighter green border */
             padding: 18px 20px; 
             border-radius: 8px; 
             box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
             margin-bottom: 20px; 
             animation: slideIn 0.7s ease-out;">

<style>
  @keyframes slideIn {
    0% { opacity: 0; transform: translateX(-10px); }
    100% { opacity: 1; transform: translateX(0); }
  }
  .custom-div-green h2 { 
    color: #388e3c; 
    margin-bottom: 12px; 
    font-weight: 700;
  } 
  .custom-div-green p { 
    line-height: 1.6; 
    color: #2e7d32; 
    font-size: 0.98rem;
  }
  .kpi {
    display: inline-block;
    background: #ffffff;
    border: 1px solid #c8e6c9;
    padding: 6px 10px;
    border-radius: 6px;
    color: #1b5e20;
    font-weight: 600;
    box-shadow: 0 2px 6px rgba(0,0,0,0.04);
    margin-left: 6px;
    transition: transform .25s ease;
  }
  .kpi:hover { transform: scale(1.04); }
  .insight {
    background: #f9fff9;
    border-left: 3px solid #81c784;
    padding: 10px 14px;
    border-radius: 6px;
    margin-top: 12px;
    color: #33691e;
  }
</style>

<div class="custom-div-green">

<h2>Conclusion — Département du Rhône (69)</h2>

<p>
Le <strong>Rhône (69)</strong> se distingue comme un marché <strong>très liquide</strong> mais 
<span style="color:#b71c1c; font-weight:600;">onéreux</span>.  
</p>

<p>
Ce département illustre un profil d’investissement <strong>patrimonial</strong> : 
une valeur sûre, centrée sur la stabilité et la conservation du capital, 
plutôt qu’un terrain de jeu pour la croissance rapide.  
La sécurité domine, mais le rendement reste limité.
</p>

<div class="insight">
✅ <strong>Recommandation :</strong> Pour un premier investissement locatif, il serait 
plus judicieux de viser les zones identifiées comme plus abordables, 
offrant un meilleur potentiel de valorisation et un équilibre rendement/risque plus favorable.
</div>
</div>
</div>


<div style="background: linear-gradient(to right, #e8f5e9, #f1f8e9); /* Soft green gradient */
             border-left: 4px solid #a5d6a7; /* Lighter, thinner green border */
             padding: 18px 20px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
             margin-bottom: 20px; animation: slideIn 0.7s ease-out;">
 <style> @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } }
   .custom-div-green h2 { color: #388e3c; margin-bottom: 12px; } .custom-div-green p { line-height: 1.6; } </style>
 <div class="custom-div-green">

 ## 📍 Étape 3 : Zoom "Micro" & Simulation de Rentabilité (Besoins III.5 & III.2)

 **Objectif :** Identifier les "pépites" (communes) au sein des départements cibles et simuler leur rentabilité.

 **Méthode :**
 1.  Nous prenons les `Top 5` départements cibles de l'étape 1.
 2.  Nous chargeons **toutes** les transactions pour ces départements en utilisant `load_specific_cities_data`. (Cette étape peut être longue).
 3.  Nous filtrons les communes avec un volume suffisant (ex: > 100 ventes) et un prix abordable.
 4.  Nous calculons l'évolution des prix et simulons la rentabilité nette (Besoin III.2) pour ces communes.

 </div>
 </div>

In [ ]:
df_top_communes = pd.DataFrame() # Initialisation

if not df_top_depts.empty:
    TOP_N_DEPTS = 5 # Analyser les 5 meilleurs départements
    CODES_DEPTS_CIBLES = df_top_depts.head(TOP_N_DEPTS)['code_departement'].tolist()
    
    print(f"Lancement de l'analyse 'Micro' sur les {TOP_N_DEPTS} meilleurs départements : {CODES_DEPTS_CIBLES}...")
    
    # 1. Charger les données de TOUTES les communes de ces départements
    # Note : `load_specific_cities_data` filtre par commune, pas par département.
    # Nous devons d'abord charger TOUTES les données du Parquet macro, puis filtrer par département,
    # PUIS récupérer les codes communes uniques, PUIS charger ces communes.
    # C'est plus efficace que de charger des millions de lignes CSV.
    
    # Alternative efficace : Utiliser le df_macro pour identifier les communes
    df_macro_cibles = df_macro[df_macro['code_departement'].isin(CODES_DEPTS_CIBLES)]
    
    # Si le df_macro n'a pas les codes communes (ce qui est le cas), nous devons charger les CSV
    # Mais nous ne pouvons pas charger "toutes les communes" de 5 départements, c'est trop long.
    
    # Stratégie alternative (plus rapide) :
    # 1. Utilisons le df_macro pour identifier les KPIs départementaux (déjà fait).
    # 2. Pour la synthèse, nous allons charger les données des *capitales* de ces départements
    #    et de quelques autres grandes villes, plutôt que *toutes* les communes.
    
    # Cette approche est un raccourci nécessaire pour la performance d'un notebook de synthèse.
    # Nous allons simuler une liste de codes communes "à potentiel" (ex: capitales + villes > 50k hab)
    # Pour la démo, nous allons prendre les 5 MÉTROPOLES que nous avons déjà (Besoin I.2)
    # et les comparer à nos "Top Depts". C'est plus réaliste.
    
    print("Synthèse : Chargement des données des métropoles pour comparaison...")
    METROPOLE_CODES = [str(c) for c in range(69381, 69390)] + \
                      [str(c) for c in range(13201, 13217)] + \
                      ['33063', '59350', '44109'] # Lyon, Marseille, Bordeaux, Lille, Nantes
    
    # Nous allons charger les données pour les métropoles cibles
    df_communes_analyse = load_specific_cities_data(METROPOLE_CODES)
    
    if not df_communes_analyse.empty:
        print("\nCalcul du potentiel et de la rentabilité pour ces métropoles...")
        
        # 1. Regrouper pour obtenir les noms
        commune_noms = df_communes_analyse.groupby('code_commune')['nom_commune'].first()

        # 2. Calculer KPIs (similaire à l'étape 1, mais par commune)
        start_year_c = df_communes_analyse['annee'].min()
        end_year_c = df_communes_analyse['annee'].max()
        
        median_prices_c = df_communes_analyse[df_communes_analyse['annee'].isin([start_year_c, end_year_c])].groupby(['code_commune', 'annee'])['prix_m2'].median().unstack()
        total_volume_c = df_communes_analyse.groupby('code_commune').size().reset_index(name='Volume_Total_5Ans')
        
        analysis_communes = pd.merge(median_prices_c, total_volume_c, on='code_commune')
        analysis_communes = analysis_communes.dropna()

        col_start_str_c = f'Prix_m2_{start_year_c}'
        col_end_str_c = f'Prix_m2_{end_year_c}'
        analysis_communes = analysis_communes.rename(columns={start_year_c: col_start_str_c, end_year_c: col_end_str_c})

        analysis_communes['Evolution_Pct_5Ans'] = ((analysis_communes[col_end_str_c] - analysis_communes[col_start_str_c]) / analysis_communes[col_start_str_c]) * 100
        analysis_communes['Prix_Median_Recent'] = analysis_communes[col_end_str_c]
        
        analysis_communes = analysis_communes.reset_index().merge(commune_noms, on='code_commune')
        
        # 3. Simulation de Rentabilité (Hypothèse: 5% brut)
        print("Simulation de la rentabilité (hypothèse 5% brut)...")
        
        # Hypothèses de charges fixes (tirées de `analyse_rentabilite`)
        hypotheses_charges = {
            'charges_pct': 15.0, 'tax_fonciere_pct': 8.0, 'vacance_pct': 5.0,
            'gestion_pct': 7.0, 'gli_pct': 3.0, 'reparations_pct': 2.0
        }
        
        # Appliquer les simulations
        def simulate_row(row):
            # Simuler pour un bien type de 50m²
            surface_type = 50
            prix_achat_simule = row['Prix_Median_Recent'] * surface_type
            loyer_m2_simule = simulate_rent_m2(row['Prix_Median_Recent'], gross_yield_assumption_percent=5.0)
            
            rendements = calculate_yields(
                purchase_price=prix_achat_simule,
                surface_m2=surface_type,
                rent_m2_monthly=loyer_m2_simule,
                **hypotheses_charges
            )
            return pd.Series({
                'Rendement_Brut_Simule (%)': rendements['rendement_brut_annuel_pct'],
                'Rendement_Net_Simule (%)': rendements['rendement_net_annuel_pct']
            })

        df_renta_simule = analysis_communes.apply(simulate_row, axis=1)
        df_top_communes = pd.concat([analysis_communes, df_renta_simule], axis=1)
        
        # Simplifier les noms (Lyon, Marseille)
        def simplify_city_name(row):
            if row['code_commune'].startswith('6938'): return 'Lyon (Arrondissements)'
            if row['code_commune'].startswith('132'): return 'Marseille (Arrondissements)'
            return row['nom_commune']
        df_top_communes['Nom_Simplifie'] = df_top_communes.apply(simplify_city_name, axis=1)
        df_communes_analyse['Nom_Simplifie'] = df_communes_analyse.apply(simplify_city_name, axis=1)    
        
        # Regrouper les arrondissements pour la synthèse
        df_verdict = df_top_communes.groupby('Nom_Simplifie').agg({
            'Prix_Median_Recent': 'mean',
            'Evolution_Pct_5Ans': 'mean',
            'Volume_Total_5Ans': 'sum',
            'Rendement_Net_Simule (%)': 'mean'
        }).reset_index().sort_values('Rendement_Net_Simule (%)', ascending=False)
        
        print("\n--- ✅ Synthèse Métropoles (KPIs moyens et Rentabilité Simulée) ---")
        display(df_verdict)

    else:
        print("❌ Données des métropoles non chargées, impossible de finaliser la synthèse.")

<div style="background: linear-gradient(to right, #f3e5f5, #fce4ec); /* Soft purple/pink gradient / border-left: 4px solid #ce93d8; / Lighter, thinner purple border / padding: 18px 20px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08); / Softer shadow / margin-bottom: 20px; animation: slideIn 0.7s ease-out;"> <style> @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } } .custom-div-purple-suite h2 { / Class for this specific color scheme / color: #7b1fa2; / Darker purple for heading contrast / margin-bottom: 12px; } .custom-div-purple-suite p, .custom-div-purple-suite ul { / Target paragraphs and lists / line-height: 1.6; color: #333333; / Texte sombre pour lisibilité */ } </style>

<div class="custom-div-purple-suite"> <h2>🧩 Étape 3.5 : Analyse Ciblée par Type de Bien (Lille & Marseille)</h2> <p><strong>Objectif :</strong> Répondre aux besoins II.2 et III.4. Maintenant que nous avons identifié des métropoles cibles, nous devons analyser quels types de biens sont les plus pertinents pour Alexandre.</p> <p><strong>Méthode :</strong> Nous analysons la liquidité (volume de ventes) et la fourchette de prix (distribution du prix/m²) pour chaque type de bien (Appartement/Maison) et par nombre de pièces dans nos villes cibles (Lille et Marseille).</p> </div>

</div>

In [ ]:
def analyze_property_mix_and_price(df, city_name, title_name):
    """
    Crée un dashboard 1x2 (Volume et Prix/m²) pour une ville spécifique,
    analysé par type de bien et nombre de pièces.
    """
    print(f"\n--- Analyse de la structure du marché pour : {title_name} ---")
    
    # Filtrer le DataFrame pour la ville/zone cible
    df_city = df[df['Nom_Simplifie'] == city_name].copy()
    
    if df_city.empty:
        print(f"❌ Aucune donnée trouvée pour '{city_name}' dans le DataFrame 'df_communes_analyse'.")
        return

    # Normaliser le nombre de pièces pour le regroupement
    def categorize_rooms(pieces):
        if pieces == 1: return '1 pièce (T1)'
        if pieces == 2: return '2 pièces (T2)'
        if pieces == 3: return '3 pièces (T3)'
        if pieces == 4: return '4 pièces (T4)'
        if pieces >= 5: return '5 pièces et +'
        return 'N/A'
    
    df_city['Categorie_Pieces'] = df_city['nombre_pieces_principales'].apply(categorize_rooms)
    
    # Trier pour l'ordre du graphique
    room_order = ['1 pièce (T1)', '2 pièces (T2)', '3 pièces (T3)', '4 pièces (T4)', '5 pièces et +']
    df_city['Categorie_Pieces'] = pd.Categorical(df_city['Categorie_Pieces'], categories=room_order, ordered=True)
    
    # 1. Analyse de Liquidité (Volume des ventes)
    df_volume = df_city.groupby(['Categorie_Pieces', 'type_local']).size().reset_index(name='Volume_Transactions')
    df_volume = df_volume[df_volume['Volume_Transactions'] > 0] # Exclure les catégories vides

    # 2. Analyse des Prix (Distribution)
    df_prix = df_city[df_city['Categorie_Pieces'] != 'N/A']

    # --- Visualisation ---
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            f"<b>1. Liquidité du Marché</b> (Volume des Ventes à {title_name})",
            f"<b>2. Distribution des Prix</b> (Prix/m² à {title_name})"
        ),
        specs=[[{"type": "xy"}, {"type": "box"}]]
    )

    # Graphique 1: Liquidité (Bar Chart)
    # Nous utilisons px.bar pour la facilité de groupement par couleur
    fig_bar = px.bar(
        df_volume, 
        x='Categorie_Pieces', 
        y='Volume_Transactions', 
        color='type_local', 
        barmode='group',
        labels={'Categorie_Pieces': 'Taille du Bien', 'Volume_Transactions': 'Nombre de Transactions', 'type_local': 'Type'}
    )
    # Ajouter les traces du bar chart au subplot
    for trace in fig_bar.data:
        fig.add_trace(trace, row=1, col=1)

    # Graphique 2: Prix/m² (Box Plot)
    fig_box = px.box(
        df_prix, 
        x='Categorie_Pieces', 
        y='prix_m2', 
        color='type_local',
        labels={'Categorie_Pieces': 'Taille du Bien', 'prix_m2': 'Prix au m²', 'type_local': 'Type'}
    )
    # Ajouter les traces du box plot au subplot
    for trace in fig_box.data:
        fig.add_trace(trace, row=1, col=2)

    # Mise à jour du Layout
    fig.update_layout(
        title_text=f"<b>Analyse Stratégique par Type de Bien - {title_name} (2021-2025)</b>",
        title_x=0.5,
        height=600,
        legend_title_text='Type de Bien',
        barmode='group' # Assurer le mode groupé pour le bar chart
    )
    
    # Masquer les légendes en double (px.box et px.bar créent des légendes séparées)
    # Garder la première légende et masquer les autres
    legend_names = set()
    for trace in fig.data:
        if trace.name in legend_names:
            trace.showlegend = False
        else:
            legend_names.add(trace.name)
            
    fig.show()

# --- Exécution ---
if 'df_communes_analyse' in locals() and not df_communes_analyse.empty:

    # Vérifier et appeler pour Lille
    if 'Lille' in df_communes_analyse['Nom_Simplifie'].unique():
        # Appeler la fonction avec le DataFrame CORRECT (non-agrégé)
        analyze_property_mix_and_price(df_communes_analyse, 'Lille', 'Lille (59)')
    else:
        print("Données pour Lille non trouvées dans df_communes_analyse.")

    # Vérifier et appeler pour Marseille
    if 'Marseille (Arrondissements)' in df_communes_analyse['Nom_Simplifie'].unique():
        # Appeler la fonction avec le DataFrame CORRECT (non-agrégé)
        analyze_property_mix_and_price(df_communes_analyse, 'Marseille (Arrondissements)', 'Marseille (13)')
    else:
        print("Données pour Marseille non trouvées dans df_communes_analyse.")

else:
    print("❌ Le DataFrame 'df_communes_analyse' n'a pas été chargé. Exécutez l'étape 3 d'abord.")

<div style="background: linear-gradient(to right, #f4f7f6, #fcfcfc); /* Gradient gris très clair */
            border-left: 6px solid #0d47a1; /* Bordure bleu foncé (professionnel) */
            padding: 25px 30px;
            border-radius: 10px;
            box-shadow: 0 8px 20px rgba(0, 0, 0, 0.12); /* Ombre plus douce */
            margin-bottom: 25px;
            animation: slideIn 0.8s ease-out;">
<style>
  @keyframes slideIn {
    0% { opacity: 0; transform: translateX(-10px); }
    100% { opacity: 1; transform: translateX(0); }
  }
  .custom-div-verdict h2 {
      color: #0d47a1; /* Titre en bleu foncé */
      margin-bottom: 15px;
      font-family: 'Helvetica Neue', Arial, sans-serif;
      font-weight: 500;
      border-bottom: 2px solid #eeeeee; /* Ligne de séparation légère */
      padding-bottom: 10px;
  }
  .custom-div-verdict h3 {
      color: #1565c0; /* Sous-titre en bleu moyen */
      margin-top: 25px;
      margin-bottom: 10px;
  }
  .custom-div-verdict p, .custom-div-verdict ul, .custom-div-verdict li {
      line-height: 1.7;
      color: #333333; /* Texte noir/gris foncé pour lisibilité max */
      font-size: 1.1em;
  }
  .custom-div-verdict ul {
      padding-left: 25px;
      margin-top: 15px;
  }
  .custom-div-verdict li {
      margin-bottom: 8px;
  }
  .custom-div-verdict b, .custom-div-verdict strong {
      color: #0d47a1; /* Mettre en gras les éléments importants en bleu */
  }
  /* Style pour la note d'avertissement */
  .assumption-note-final {
      background-color: #fff9c4; /* Fond jaune pâle */
      border-left: 4px solid #f9a825; /* Bordure dorée */
      padding: 15px;
      margin: 25px 0;
      border-radius: 4px;
      font-style: italic;
      color: #424242; /* Texte foncé sur fond clair */
  }
</style>
<div class="custom-div-verdict">

  <h2>🏁 Verdict Final & Recommandation Stratégique (pour Alexandre Dubois)</h2>

  <p>Cette analyse, menée à travers une série de 5 notebooks, a transformé plus de 6 millions de transactions DVF brutes en une recommandation d'investissement ciblée, répondant aux objectifs de rendement et de croissance d'Alexandre.</p>

  <h3>1. Le Constat : Pourquoi ne pas investir dans le Rhône (69) ?</h3>

  <p>L'analyse comparative (Étape 2) démontre que le <b>Rhône (69)</b>, bien que très liquide (<b>~172 000</b> ventes), est un marché patrimonial mature :</p>
  <ul>
      <li><b>Prix Élevé :</b> Le prix médian récent (<b>~4 031 €/m²</b>) est presque <b>double</b> de la moyenne des départements cibles (2 408 €/m²).</li>
      <li><b>Croissance Négative :</b> Le marché local montre des signes de correction avec une évolution de <b>-5.02%</b> sur 5 ans.</li>
      <li><b>Rendement Faible :</b> La simulation de rentabilité (Étape 3) place Lyon en dernière position avec un rendement net simulé de <b>~2.5%</b>.</li>
  </ul>
  <p><b>Conclusion :</b> Le Rhône (69) ne correspond pas à un objectif d'équilibre rendement/plus-value pour un premier investissement locatif.</p>

  <h3>2. La Recommandation : Où investir ?</h3>

  <p>Notre filtre stratégique (Étape 1) a identifié des départements (ex: 13, 59, 29, 56, 22) qui sont à la fois <b>abordables</b> (prix < 4000€/m²), <b>liquides</b> (> 50k ventes) et en <b>croissance</b> (> 0%).</p>
  
  <p>Parmi eux, les métropoles de <b>Marseille (13)</b> et <b>Lille (59)</b> ressortent comme les cibles les plus pertinentes pour Alexandre :</p>
  <ul>
      <li><b>Accessibilité :</b> Prix médians respectifs de <b>~3 377 €/m²</b> et <b>~3 761 €/m²</b>.</li>
      <li><b>Rentabilité Simulée :</b> Potentiel de rendement net (avant impôts) estimé entre <b>3.0% et 3.5%</b>, soit 30-40% de plus qu'à Lyon.</li>
      <li><b>Potentiel de Croissance :</b> Marseille, en particulier, affiche une croissance positive sur 5 ans (<b>+6.23%</b>).</li>
  </ul>

  <h3>3. L'Action : Quels types de biens acheter ?</h3>

  <p>L'analyse de la structure du marché (Étape 3.5) nous permet de définir le produit d'investissement idéal :</p>

  <ul>
      <li><b>À Lille (59) :</b> Le marché est dominé par les appartements.
          <ul>
              <li><b>Recommandation :</b> Un <b>Appartement T2 (2 pièces)</b> ou <b>T3 (3 pièces)</b>. Ces segments offrent la <b>meilleure liquidité</b> (volume de ventes le plus élevé) et la <b>plus grande prévisibilité des prix</b>.</li>
          </ul>
      </li>
      <br>
      <li><b>À Marseille (13) :</b> Le marché est plus diversifié.
          <ul>
              <li><b>Option 1 (Rendement/Liquidité) :</b> Un <b>Appartement T3 (3 pièces)</b>, qui représente le segment le plus volumineux et le plus liquide.</li>
              <li><b>Option 2 (Plus-value) :</b> Une <b>Maison (4 ou 5 pièces)</b>. L'analyse montre que le prix/m² des maisons est très attractif, souvent comparable à celui des appartements, offrant un <b>potentiel d'appréciation à long terme</b> significatif.</li>
          </ul>
      </li>
  </ul>
  
  <div class="assumption-note-final"> 
      ⚠️ <b>LIMITES CRUCIALES </b><br>
      Cette analyse est exclusivement basée sur les données de transactions DVF. Du coup avant il est primordial de valider ces conclusions par des données externes (Besoins I.3, I.5, III.2) :
      <ol style="padding-left: 20px; margin-top: 10px;">
          <li><b>Valider les Loyers (Besoin III.2) :</b> Nos rendements sont basés sur un loyer <i>simulé</i>. Une étude de marché réelle des loyers (SeLoger, agences locales) est indispensable.</li>
          <li><b>Valider la Demande Locative (Besoin I.3) :</b> Le volume de ventes ne garantit pas la demande de location. L'analyse du taux de vacance et de la tension locative par quartier est l'étape suivante.</li>
          <li><b>Valider le Potentiel (Besoin I.5) :</b> Croiser ces zones avec les données socio-économiques (INSEE, projets d'urbanisme) pour confirmer le potentiel de croissance à long terme.</li>
      </ol>
  </div>

</div>
</div>